In [4]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver('SolveSimpleSystem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

In [5]:
food = [['ham',650, 4],
 ['lettuce',70,1.5],
 ['cheese',1670,5],
 ['tuna',830,20],
 ['bread',1300,1.20]]

In [6]:
#food is a list of groceries, calories and prices
variable_list = [[]] * len(food)
for i in range(0, len(food)):
    #you must buy at least minShop of each
    variable_list[i] = solver.NumVar(4/16.0, solver.infinity(), str(food[i][0]))

In [7]:
variable_list

[ham, lettuce, cheese, tuna, bread]

In [8]:
#same thing but with comprehension
# variable_list=[solver.NumVar(minShop, solver.infinity(), str(food[i][0])) for i in range(0, len(food))]

In [9]:
# Define the constraints   
constraint_list=[]
# Constraint 1: totalWeight<maxWeight
# ham + lettuce + cheese + tuna + bread <= maxWeight
constraint_list.append(solver.Constraint(0, 10))
for i in range(0, len(food)):
    constraint_list[0].SetCoefficient(variable_list[i],1)

# Constraint 2: totalPrice<=maxCost
constraint_list.append(solver.Constraint(0, 100))
for i in range(0, len(food)):
    constraint_list[1].SetCoefficient(variable_list[i],food[i][2])

# Constraint 3: totalCalories>=minCals
constraint_list.append(solver.Constraint(14000, 14000 + 100))
for i in range(0, len(food)):
    constraint_list[2].SetCoefficient(variable_list[i],food[i][1])

In [10]:
objective = solver.Objective()

In [11]:
# Define our objective: maximizing calories
for i in range(0, len(food)):
    objective.SetCoefficient(variable_list[i], food[i][1])
objective.SetMaximization()

In [12]:
result_status = solver.Solve()
 
if result_status == solver.OPTIMAL:
    print('Successful solve.')
    # The problem has an optimal solution.
    print(('Problem solved in %f milliseconds' % solver.wall_time()))
    # The objective value of the solution.
    print(('Optimal objective value = %f' % solver.Objective().Value()))
    # The value of each variable in the solution.
    var_sum=0
    for variable in variable_list:
        print(('%s = %f' % (variable.name(), variable.solution_value())))
        var_sum+=variable.solution_value()
    print(('Variable sum = %f' % var_sum));
 
    print('Advanced usage:')
    print(('Problem solved in %d iterations' % solver.iterations()))
 
    for variable in variable_list:
        print(('%s: reduced cost = %f' % (variable.name(), variable.reduced_cost())))
     
    activities = solver.ComputeConstraintActivities()
    for i, constraint in enumerate(constraint_list):
        print(('constraint %d: dual value = %f\n'
          '               activity = %f' %
          (i, constraint.dual_value(), activities[constraint.index()])))
 
elif result_status == solver.INFEASIBLE:
    print('No solution found.')
elif result_status == solver.POSSIBLE_OVERFLOW:
    print('Some inputs are too large and may cause an integer overflow.')
 

Successful solve.
Problem solved in 51809.000000 milliseconds
Optimal objective value = 14100.000000
ham = 0.250000
lettuce = 0.250000
cheese = 4.560811
tuna = 0.250000
bread = 4.689189
Variable sum = 10.000000
Advanced usage:
Problem solved in 3 iterations
ham: reduced cost = 0.000000
lettuce: reduced cost = 0.000000
cheese: reduced cost = 0.000000
tuna: reduced cost = 0.000000
bread: reduced cost = 0.000000
constraint 0: dual value = -0.000000
               activity = 10.000000
constraint 1: dual value = -0.000000
               activity = 34.806081
constraint 2: dual value = 1.000000
               activity = 14100.000000
